## Imports


In [1]:
import json
import pandas as pd
import os
from textblob import TextBlob
import nltk
#import tweepy
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
#from gensim import corpora
#from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
#from nltk.stem.porter import PorterStemmer
#from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objs as go

In [2]:
pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.


In [3]:
print(os.getcwd())

/home/reichardtma/code/reichardtma/prestweets/notebooks


## Loading Data

In [4]:
csv_path = os.path.join("..","raw_data")

In [5]:
pd.read_csv(os.path.join(csv_path, 'borisjohnson_tweets.csv')).head(2)

,Unnamed: 0,id,url,date,content,hashtags,cashtags,media,lang
0,0,1506711585468911621,https://twitter.com/BorisJohnson/status/150671...,2022-03-23 19:16:49+00:00,In 2019 I promised to equalise National Insura...,['SpringStatement2022'],NaN,NaN,en
1,1,1506684769559621633,https://twitter.com/BorisJohnson/status/150668...,2022-03-23 17:30:16+00:00,I spoke to President @ZelenskyyUa and reiterat...,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.Photo', '...",en


In [6]:
boris = pd.read_csv(os.path.join(csv_path, 'borisjohnson_tweets.csv'))
jokowi = pd.read_csv(os.path.join(csv_path, 'jokowi_tweets.csv'))
jokowi['content'] = jokowi['Translation']
jokowi = jokowi.drop(columns= ['Translation'])
jb = pd.read_csv(os.path.join(csv_path, 'jairbolsonaro_tweets.csv'))
jb['content'] = jb['Translation']
jb = jb.drop(columns= ['Translation'])
macron = pd.read_csv(os.path.join(csv_path, 'EmmanuelMacron_tweets.csv'))
macron['content'] = macron['Translation']
macron = macron.drop(columns= ['Translation'])
cr = pd.read_csv(os.path.join(csv_path, 'CyrilRamaphosa_tweets.csv'))

In [7]:
presis = [boris,jokowi,jb,macron,cr]
presis_name = ['boris','jokowi','jb','macron','cr']
for i in range(5):
    presis[i]['Unnamed: 0'] = presis_name[i]

In [8]:
jokowi.head()

,Unnamed: 0,id,url,date,content,hashtags,cashtags,media,lang
0,jokowi,1506625755173642247,https://twitter.com/jokowi/status/150662575517...,2022-03-23 13:35:46+00:00,The situation of the Covid-19 pandemic in Indo...,NaN,NaN,NaN,in
1,jokowi,1506620313672380417,https://twitter.com/jokowi/status/150662031367...,2022-03-23 13:14:09+00:00,I'm heading to East Nusa Tenggara tonight for ...,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.Photo', '...",in
2,jokowi,1506547045179351045,https://twitter.com/jokowi/status/150654704517...,2022-03-23 08:23:00+00:00,Today I received the general chairmen of 12 st...,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.Photo', '...",in
3,jokowi,1506520169694187520,https://twitter.com/jokowi/status/150652016969...,2022-03-23 06:36:12+00:00,One of the largest handicraft product exhibiti...,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.Photo', '...",in
4,jokowi,1506520163289423872,https://twitter.com/jokowi/status/150652016328...,2022-03-23 06:36:11+00:00,"Two years of vacuum due to the pandemic, INACR...",NaN,NaN,"[{'_type': 'snscrape.modules.twitter.Photo', '...",in


In [9]:
pres = pd.concat([boris,jokowi,jb,macron,cr], axis=0)

In [10]:
pres = pres.rename(columns={"Unnamed: 0": "name"})

In [11]:
pres.content.describe()

count           39241
unique          39101
top       @secomvc 🇧🇷
freq                8
Name: content, dtype: object

## Cleaning Data

In [12]:
def remove_punctuation(text):
    text = ''.join([i for i in text if i not in string.punctuation])
    return text
def lower_case(text):
    text = text.lower()
    return text
def remove_numbers(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    text = ''.join(lemmatizer.lemmatize(word) for word in text)
    return text
def stop_words(text):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text) 
    text = ' '.join(w for w in word_tokens if not w in stop_words)
    return text

pres = pres[pres.content.map(lambda x: isinstance(x,str))]
    
pres['clean_translation'] = pres['content'].apply(lemmatize).\
apply(lower_case).apply(remove_numbers).apply(remove_punctuation).apply(stop_words)
pres.head()

,name,id,url,date,content,hashtags,cashtags,media,lang,clean_translation
0,boris,1506711585468911621,https://twitter.com/BorisJohnson/status/150671...,2022-03-23 19:16:49+00:00,In 2019 I promised to equalise National Insura...,['SpringStatement2022'],NaN,NaN,en,promised equalise national insurance income ta...
1,boris,1506684769559621633,https://twitter.com/BorisJohnson/status/150668...,2022-03-23 17:30:16+00:00,I spoke to President @ZelenskyyUa and reiterat...,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.Photo', '...",en,spoke president zelenskyyua reiterated uk ’ ho...
2,boris,1506675316273692678,https://twitter.com/BorisJohnson/status/150667...,2022-03-23 16:52:42+00:00,Those lost to Covid will never be out of our h...,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.Photo', '...",en,lost covid never hearts minds today reflect na...
3,boris,1506647014209200130,https://twitter.com/BorisJohnson/status/150664...,2022-03-23 15:00:14+00:00,We're cutting taxes for hardworking people:\n ...,NaN,NaN,NaN,en,cutting taxes hardworking people matching nics...
4,boris,1506588800868392962,https://twitter.com/BorisJohnson/status/150658...,2022-03-23 11:08:55+00:00,The UK &amp; Ireland bid for the UEFA Euro 202...,NaN,NaN,NaN,en,uk amp ireland bid uefa euro government ’ full...


In [13]:
pres['date'] = pd.to_datetime(pres['date'])
pres.date.min()

Timestamp('2010-04-01 01:33:17+0000', tz='UTC')

In [14]:
presidents_ = list(pres['name'].unique())
presidents_

['boris', 'jokowi', 'jb', 'macron', 'cr']

### Creating individual dataframes and setting datetime

In [15]:
boris_df = pres[pres['name'] == 'boris']
#boris_df = boris_df[boris_df.polarity != 0]
#boris_df = boris_df[boris_df.objectivity != 0]
print(len(boris_df))

jokowi_df = pres[pres['name'] == 'jokowi']
#jokowi_df = jokowi_df[jokowi_df.polarity != 0]
#jokowi_df = jokowi_df[jokowi_df.objectivity != 0]
print(len(jokowi_df))

jb_df = pres[pres['name'] == 'jb']
#jb_df = jb_df[jb_df.polarity != 0]
#jb_df = jb_df[jb_df.objectivity != 0]
print(len(jb_df))

macron_df = pres[pres['name'] == 'macron']
#macron_df = macron_df[macron_df.polarity != 0]
#macron_df = macron_df[macron_df.objectivity != 0]
print(len(macron_df))

cr_df = pres[pres['name'] == 'cr']
#cr_df = cr_df[cr_df.polarity != 0]
#cr_df = cr_df[cr_df.objectivity != 0]
print(len(cr_df))

4332
4080
11610
10417
8802


In [27]:
boris_by_date = pd.Series(boris_df['date'])
boris_by_date = boris_by_date.dt.to_period('D').value_counts().sort_index()
boris_by_date.index = pd.PeriodIndex(boris_by_date.index)
boris_df_date = boris_by_date.rename_axis('day').reset_index(name='counts')
boris_df_date

/home/reichardtma/.pyenv/versions/3.8.12/envs/prestweets/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,day,counts
0,2015-04-01,2
1,2015-04-02,28
2,2015-04-03,1
3,2015-04-04,1
4,2015-04-05,2
...,...,...
1548,2022-03-18,4
1549,2022-03-20,3
1550,2022-03-21,4
1551,2022-03-22,4


In [28]:
jokowi_by_date = pd.Series(jokowi_df['date'])
jokowi_by_date = jokowi_by_date.dt.to_period('D').value_counts().sort_index()
jokowi_by_date.index = pd.PeriodIndex(jokowi_by_date.index)
jokowi_df_date = jokowi_by_date.rename_axis('day').reset_index(name='counts')
jokowi_df_date

/home/reichardtma/.pyenv/versions/3.8.12/envs/prestweets/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,day,counts
0,2015-06-20,1
1,2015-06-22,1
2,2015-06-30,2
3,2015-07-02,1
4,2015-07-04,14
...,...,...
1697,2022-03-18,2
1698,2022-03-20,9
1699,2022-03-21,2
1700,2022-03-22,2


### Plotting amount of tweets

In [40]:
fig = go.Figure(data=go.Scatter(x=boris_df_date['day'].astype(dtype=str), 
                        y=boris_df_date['counts'],
                        marker_color='indianred', text="counts", name="Boris Johnson"))

fig = fig.add_trace(go.Scatter(x=jokowi_df_date['day'].astype(dtype=str), 
                        y=jokowi_df_date['counts'],
                        marker_color='blue', text="counts", name="Joko Widodo"))

fig.update_layout({"title": 'Tweets from Boris Johnson and Joko Widodo',
                   "xaxis": {"title":"Time"},
                   "yaxis": {"title":"Total tweets"},
                   "showlegend": True})
#fig.write_image("by-month.png",format="png", width=1000, height=600, scale=3)
fig.show()

## Analyzing Sentiment

In [41]:
def polarity(s):
    pol = TextBlob(s)
    pol = pol.sentiment[0]
    return pol

def subjectivity(s):
    obj = TextBlob(s)
    obj = obj.sentiment[1]
    return obj

In [96]:
presidents = [boris_df, jokowi_df, jb_df, macron_df, cr_df]

for president in presidents:

    president['polarity'] = president['clean_translation'].apply(polarity)
    president['subjectivity'] = president['clean_translation'].apply(subjectivity)

/tmp/ipykernel_5535/285577367.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_5535/285577367.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [52]:
[pres['name'].unique()]

[array(['boris', 'jokowi', 'jb', 'macron', 'cr'], dtype=object)]

In [54]:
sentiment_df = {'polarity': [president['polarity'].mean() for president in presidents],\
                'subjectivity': [president['subjectivity'].mean() for president in presidents]}

sentiment_df = pd.DataFrame(data=sentiment_df)
sentiment_df.insert(0, "president", pres['name'].unique())
sentiment_df

,president,polarity,subjectivity
0,boris,0.175356,0.432986
1,jokowi,0.138427,0.381524
2,jb,0.071982,0.250590
3,macron,0.084495,0.266946
4,cr,0.104043,0.344474


In [56]:
fig = px.scatter(sentiment_df, x="polarity", y="subjectivity", color=sentiment_df.president)
fig.show()

## Topic Analysis

In [103]:
for president in presidents:
    print(president['name'].unique()[0]+":")
    vectorizer = TfidfVectorizer().fit(president['clean_translation'])
    data_vectorized = vectorizer.transform(president['clean_translation'])
    model = LatentDirichletAllocation(n_components=3).fit(data_vectorized)
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        print("")

boris:
Topic 0:
[('uk', 53.20836795142034), ('people', 47.51712814514341), ('thank', 44.27330133136799), ('today', 41.22208238242334), ('nhs', 37.857084406220444), ('country', 36.88026907123859), ('work', 36.17346733703428), ('together', 34.18558129357036), ('amp', 33.607278664163644), ('us', 33.38797418269792)]

Topic 1:
[('get', 68.6154221284451), ('brexit', 67.17122663574948), ('done', 65.87147667829474), ('conservative', 29.78858194288894), ('live', 28.464566497790134), ('vote', 28.031387824609514), ('watch', 27.76648194505311), ('getbrexitdone', 25.551512051140385), ('let', 25.25262577084231), ('country', 23.676055116451252)]

Topic 2:
[('amp', 34.23875772973911), ('eu', 19.301987547824325), ('uk', 17.50159221564025), ('today', 17.2182570519891), ('security', 14.857205491391719), ('talks', 14.666451537274964), ('new', 13.014117605168122), ('great', 12.61465079117798), ('trade', 12.590670932548223), ('foreign', 11.785831111628019)]

jokowi:
Topic 0:
[('jkw', 46.10241329697002), ('i